In [ ]:
!pip install transformers accelerate peft datasets bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset

# Load the datasets
train_dataset = load_dataset('csv', data_files='/content/drive/MyDrive/BTP/BTP_LLM-20241106T104614Z-001/BTP_LLM/train_data.csv')['train']
val_dataset = load_dataset('csv', data_files='/content/drive/MyDrive/BTP/BTP_LLM-20241106T104614Z-001/BTP_LLM/test_data.csv')['train']
test_dataset = load_dataset('csv', data_files='/content/drive/MyDrive/BTP/BTP_LLM-20241106T104614Z-001/BTP_LLM/val_data.csv')['train']

train_dataset = train_dataset.shuffle(seed=42).select(range(10000))
val_dataset = val_dataset.shuffle(seed=42).select(range(1500))
test_dataset = test_dataset.shuffle(seed=42).select(range(2000))

# Inspecting a sample data to check structure
print(train_dataset[0])
print(test_dataset[0])
print(val_dataset[0])


print("Number of samples in train dataset:", len(train_dataset))
print("Number of samples in test dataset:", len(test_dataset))
print("Number of samples in validation dataset:", len(val_dataset))



Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

{'recipe_name': 'Big Sky Beef', 'ingredients': '["1 lb. Chuckwagon Ground Beef", "8 oz. can tomato sauce", "1/2 c. onions, chopped", "16 oz. can kidney beans", "1 pkg. taco seasoning mix", "1 c. water", "1-1/2 c. shredded cheddar cheese", "2 c. Fritos Corn Chips"]', 'instructions': '["Brown the ground beef and onion; drain. Mix in the taco seasoning mix and water, bring to a boil. Reduce heat. Add tomato sauce and kidney beans; simmer for 10 minutes. Spread 1 cup of the corn chips in baking dish. Layer with half of the meat mixture. Next layer with half of the cheddar cheese. Spread second cup of corn chips and repeat layers. Cover and bake at 350 degrees for 30 minutes. Serve with salsa and sour cream."]'}
{'recipe_name': 'Applesauce Fruit Cake', 'ingredients': '["1 c. butter", "1 c. white sugar", "1 c. brown sugar", "2 eggs", "4 c. sifted flour", "2 c. raisins", "1/2 c. chopped candied cherries", "1 c. candied pineapple", "1 c. candied fruit mix", "1 c. nut meats", "1 tsp. soda", "2 

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

login(token="Add your huggingface token here")

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the LLAMA 3.1 model and tokenizer with 4-bit quantization
# llama_model_path = "meta-llama/Meta-Llama-3-8B-Instruct"

llama_model_path = "microsoft/phi-2"

llama_tokenizer = AutoTokenizer.from_pretrained(llama_model_path)
# llama_tokenizer = llama_tokenizer.eos
llama_model = AutoModelForCausalLM.from_pretrained(
    llama_model_path,
    quantization_config=quantization_config
)
# Prepare the model for QLoRA training
model = prepare_model_for_kbit_training(llama_model)

# Set up QLoRA configuration
lora_config = LoraConfig(
    r=4,  # Lowered from 8 for faster training
    lora_alpha=16,  # Lowered alpha value
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


# Apply QLoRA to the model
model = get_peft_model(model, lora_config)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
from transformers import Trainer, TrainingArguments

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
# from transformers import Trainer, TrainingArguments, TrainerCallback
# import math

# llama_tokenizer.pad_token = llama_tokenizer.eos_token

# # Custom callback to save the model every 0.1 epochs
# class SaveEveryFractionalEpochCallback(TrainerCallback):
#     def __init__(self, save_steps):
#         self.save_steps = save_steps

#     def on_step_end(self, args, state, control, **kwargs):
#         # Save every 0.1 epoch
#         if state.global_step % self.save_steps == 0:
#             output_dir = f"{args.output_dir}/checkpoint-{state.global_step}"
#             kwargs["model"].save_pretrained(output_dir)
#             kwargs["tokenizer"].save_pretrained(output_dir)
#             print(f"Model saved at {output_dir}")

# # Function to process the dataset
# def preprocess_function(examples):
#     # Concatenate ingredients into a single string
#     ingredients_text = [" ".join(ing_list) for ing_list in examples["ingredients"]]

#     # Tokenize the ingredients text
#     inputs = llama_tokenizer(ingredients_text, padding="max_length", truncation=True, max_length=512)
#     labels = llama_tokenizer(examples["instructions"], padding="max_length", truncation=True, max_length=512)

#     # Set labels as expected outputs
#     inputs["labels"] = labels["input_ids"]

#     return inputs

# # Tokenize dataset
# train_tokenized = train_dataset.map(preprocess_function, batched=True)
# val_tokenized = val_dataset.map(preprocess_function, batched=True)


# # Define training arguments
# training_args = TrainingArguments(
#     output_dir="/content/drive/MyDrive/BTP/Phi_model",  # Save model to your specified path
#     evaluation_strategy="epoch",
#     learning_rate=3e-3,  # Higher learning rate for faster convergence
#     per_device_train_batch_size=2,
#     per_device_eval_batch_size=4,
#     gradient_accumulation_steps=4,  # Reduce to accelerate training
#     num_train_epochs=1,  # Start with 1 epoch, monitor results
#     weight_decay=0.01,
#     logging_steps=20,
#     save_strategy="no",  # Disable default saving
#     fp16=True,  # Mixed precision
#     max_steps=5000,  # Limit training steps to control time further
# )

# # Calculate steps for 0.1 epoch saving frequency
# total_steps = len(train_tokenized) // training_args.per_device_train_batch_size * training_args.num_train_epochs
# save_steps = math.ceil(total_steps * 0.1)


# # Initialize the Trainer with adjusted settings and custom callback
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_tokenized,
#     eval_dataset=val_tokenized,
#     callbacks=[SaveEveryFractionalEpochCallback(save_steps=save_steps)]
# )

# # Train the model
# trainer.train()


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
from transformers import Trainer, TrainingArguments, TrainerCallback
import math

# Ensure the tokenizer has a proper pad token
llama_tokenizer.pad_token = llama_tokenizer.eos_token

# Custom callback to save the model every 0.1 epochs
class SaveEveryFractionalEpochCallback(TrainerCallback):
    def __init__(self, save_steps, tokenizer):
        self.save_steps = save_steps
        self.tokenizer = tokenizer  # Store the tokenizer

    def on_step_end(self, args, state, control, **kwargs):
        # Save every 0.1 epoch
        if state.global_step % self.save_steps == 0 and state.global_step > 0:
            output_dir = f"{args.output_dir}/checkpoint-{state.global_step}"
            kwargs["model"].save_pretrained(output_dir)
            self.tokenizer.save_pretrained(output_dir)  # Use the stored tokenizer
            print(f"Model saved at {output_dir}")

# Function to process the dataset
def preprocess_function(examples):
    # Concatenate ingredients into a single string
    ingredients_text = [" ".join(ing_list) for ing_list in examples["ingredients"]]

    # Tokenize the ingredients text
    inputs = llama_tokenizer(ingredients_text, padding="max_length", truncation=True, max_length=256)
    labels = llama_tokenizer(examples["instructions"], padding="max_length", truncation=True, max_length=256)

    # Set labels as expected outputs
    inputs["labels"] = labels["input_ids"]

    return inputs

# Tokenize dataset with multiple workers for faster preprocessing
train_tokenized = train_dataset.map(preprocess_function, batched=True, num_proc=4)
val_tokenized = val_dataset.map(preprocess_function, batched=True, num_proc=4)

# Define training arguments with optimized settings
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/BTP/Phi_model",
    evaluation_strategy="epoch",
    learning_rate=3e-3,
    per_device_train_batch_size=4,  # Increased batch size
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,  # Reduced for faster updates
    num_train_epochs=2,  # Adjusted number of epochs
    weight_decay=0.01,
    logging_steps=50,  # Less frequent logging for reduced I/O overhead
    save_strategy="no",  # Custom saving strategy
    fp16=True,  # Mixed precision
    max_steps=5000,
    dataloader_num_workers=4,  # Speed up dataloading
)

# Calculate steps for 0.1 epoch saving frequency
total_steps = len(train_tokenized) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)
save_steps = math.ceil(total_steps * 0.1)

# Initialize the Trainer with adjusted settings and custom callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    callbacks=[SaveEveryFractionalEpochCallback(save_steps=save_steps, tokenizer=llama_tokenizer)]
)

# Train the model
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_r

Epoch,Training Loss,Validation Loss
1,2.185000,2.064477
2,2.117100,2.074352


Model saved at /content/drive/MyDrive/BTP/Phi_model/checkpoint-125
Model saved at /content/drive/MyDrive/BTP/Phi_model/checkpoint-250
Model saved at /content/drive/MyDrive/BTP/Phi_model/checkpoint-375
Model saved at /content/drive/MyDrive/BTP/Phi_model/checkpoint-500
Model saved at /content/drive/MyDrive/BTP/Phi_model/checkpoint-625
Model saved at /content/drive/MyDrive/BTP/Phi_model/checkpoint-750
Model saved at /content/drive/MyDrive/BTP/Phi_model/checkpoint-875
Model saved at /content/drive/MyDrive/BTP/Phi_model/checkpoint-1000
Model saved at /content/drive/MyDrive/BTP/Phi_model/checkpoint-1125
Model saved at /content/drive/MyDrive/BTP/Phi_model/checkpoint-1250
Model saved at /content/drive/MyDrive/BTP/Phi_model/checkpoint-1375
Model saved at /content/drive/MyDrive/BTP/Phi_model/checkpoint-1500
Model saved at /content/drive/MyDrive/BTP/Phi_model/checkpoint-1625
Model saved at /content/drive/MyDrive/BTP/Phi_model/checkpoint-1750
Model saved at /content/drive/MyDrive/BTP/Phi_model/che

In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=b596cedd2cb7d1bb56bc2aa6b35cbce7630c504dc4e4a3ecabbd1e3a19fd729f
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import torch
import json

# Load the fine-tuned model and tokenizer
model_path = "/content/drive/MyDrive/BTP/Phi_model/checkpoint-3500"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
def generate_instructions(ingredients):
    """Generate recipe instructions from ingredients using the fine-tuned model."""
    prompt = f"Given the ingredients: {ingredients}, provide step-by-step cooking instructions."
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=256).to("cuda")
    model.to("cuda")

    with torch.no_grad():
        output = model.generate(**inputs, max_length=256, num_return_sequences=1, do_sample=True, temperature=0.7)

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text


In [ ]:
bleu_scores = []
rouge_scores = []
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

for i, sample in enumerate(test_dataset):
    ingredients = json.loads(sample["ingredients"])  # Convert string to list
    ingredients_text = ", ".join(ingredients)  # Convert list to string
    reference_instructions = " ".join(json.loads(sample["instructions"]))  # Convert list to string

    generated_instructions = generate_instructions(ingredients_text)

    # Compute BLEU score
    reference_tokens = [reference_instructions.split()]
    generated_tokens = generated_instructions.split()
    bleu = sentence_bleu(reference_tokens, generated_tokens)
    bleu_scores.append(bleu)

    # Compute ROUGE score
    rouge = scorer.score(reference_instructions, generated_instructions)["rougeL"].fmeasure
    rouge_scores.append(rouge)

    # Print some sample outputs
    if i < 5:
        print(f"Recipe: {sample['recipe_name']}")
        print(f"Ingredients: {ingredients_text}")
        print(f"Generated Instructions: {generated_instructions}")
        print(f"Reference Instructions: {reference_instructions}")
        print(f"BLEU Score: {bleu:.4f}, ROUGE Score: {rouge:.4f}\n")

# Compute average scores
avg_bleu = sum(bleu_scores) / len(bleu_scores)
avg_rouge = sum(rouge_scores) / len(rouge_scores)
print(f"Average BLEU Score: {avg_bleu:.4f}")
print(f"Average ROUGE Score: {avg_rouge:.4f}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consid

Recipe: Applesauce Fruit Cake
Ingredients: 1 c. butter, 1 c. white sugar, 1 c. brown sugar, 2 eggs, 4 c. sifted flour, 2 c. raisins, 1/2 c. chopped candied cherries, 1 c. candied pineapple, 1 c. candied fruit mix, 1 c. nut meats, 1 tsp. soda, 2 tsp. baking powder, 3 tsp. cinnamon, 1 tsp. ground cloves, 1 tsp. ground ginger, 1 tsp. vanilla
Generated Instructions: Given the ingredients: 1 c. butter, 1 c. white sugar, 1 c. brown sugar, 2 eggs, 4 c. sifted flour, 2 c. raisins, 1/2 c. chopped candied cherries, 1 c. candied pineapple, 1 c. candied fruit mix, 1 c. nut meats, 1 tsp. soda, 2 tsp. baking powder, 3 tsp. cinnamon, 1 tsp. ground cloves, 1 tsp. ground ginger, 1 tsp. vanilla, provide step-by-step cooking instructions.PressMatchBlWellMatchGreSumCLightMixHypCasComeGreBlCombHeatWhiteDeepCGreGreGreCloseEngGreLightBreakHypCombBrownPressGreGreLetDGreBaseCutMixSlGreCombCrPushPressBuildBrownGraCCasCasGraDecGreSaMixCrFMGreCombSlWhiteMixFCasDWhiteMixBaseSpreadBrownWellSwBreakGreGreCombSaGreTrG

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Recipe: Taco Casserole
Ingredients: 1 pkg. hamburger, 1 medium onion, chopped, 1 can diced green chili pepper, 1 can tomato soup, 1 can cream of chicken soup, 1 can spicy refried beans (all cans of the same size), 2 small jalapeno peppers, diced, 1 large bag Doritos, tortilla chips or Fritos, 1/2 lb. Cheddar cheese, shredded, 3/4 lb. Mozzarella cheese, shredded, 1 pkg. taco seasoning mix or 2 Tbsp. chili powder
Generated Instructions: Given the ingredients: 1 pkg. hamburger, 1 medium onion, chopped, 1 can diced green chili pepper, 1 can tomato soup, 1 can cream of chicken soup, 1 can spicy refried beans (all cans of the same size), 2 small jalapeno peppers, diced, 1 large bag Doritos, tortilla chips or Fritos, 1/2 lb. Cheddar cheese, shredded, 3/4 lb. Mozzarella cheese, shredded, 1 pkg. taco seasoning mix or 2 Tbsp. chili powder, provide step-by-step cooking instructions.CrDigHeatHydCombMixWellCombMixGreHypBaseWellGreMixGlGreSlWhiteSquCombGreGreScInCloseCombGreMixGreMixMixMixMixHydMixD

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Recipe: Spaghetti Pizza Deluxe
Ingredients: 1 pkg. spaghetti, 1/2 c. skim milk, 1 egg, 1/2 lb. hamburger, 1 onion, 1 tsp. crushed garlic, 1 can tomato sauce, 1/4 tsp. black pepper, 1/2 c. mushrooms, 2 c. shredded Mozzarella cheese
Generated Instructions: Given the ingredients: 1 pkg. spaghetti, 1/2 c. skim milk, 1 egg, 1/2 lb. hamburger, 1 onion, 1 tsp. crushed garlic, 1 can tomato sauce, 1/4 tsp. black pepper, 1/2 c. mushrooms, 2 c. shredded Mozzarella cheese, provide step-by-step cooking instructions.DecWhiteGraGreGreHypPredBrownCombHypGreBlBrownPushGreDecCWhiteGreGreGreGreGreGreGreGreBreakCombWellWhiteMatchGreGreMixGreGreHypWhiteInMixHypMixMixSaGreDCrGreCGreCCBaseGreFCombGreMixTrCasTrackCombFLightHydMixCombCombBaseBaseSandGreGreGreMixCutMixGreCrBreakDecCrGreBaseTrMPrefCombCasMixCasGreGreDigMixMixDigCombMixMixGreGreSaGreGreCasMixGreSaMixEngMixMixMixSaPutSaMixSaSaBreakTrSaLightSaSaGreSaMatchBrownMixGreThMixPutMixSaSaMixMixRefSaSaSaGreFMixSaSaSaGreSaSaGreMixBlMixSaCutMixMixMix*MixMixMi

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-bed765a432ac>", line 10, in <cell line: 0>
    generated_instructions = generate_instructions(ingredients_text)
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-8-8c3f9c9bd8d9>", line 8, in generate_instructions
    output = model.generate(**inputs, max_length=256, num_return_sequences=1, do_sample=True, temperature=0.7)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py", line 2255, in generate
    result = self._sampl

TypeError: object of type 'NoneType' has no len()